In [1]:
from pathlib import Path
import pandas as pd
import sys
sys.path.append(str(Path.cwd().parent))

In [2]:
from scripts.config import paths, states_abbrs

In [3]:
DATA_YEAR = "2020"
FMR_EXT = "xlsx"

In [4]:
fmr_dir = paths["data"] / 'fmr'

In [5]:
print(f"FMR data:")
sources = {}
for f_obj in fmr_dir.glob(f"*.{FMR_EXT}"):
    if f_obj.name.find(DATA_YEAR) != -1 and f_obj.name.find("~") == -1:
        print(f"- {f_obj.name}")
        if f_obj.name.find("CHIP") > -1:
            label = f"fmr-chip"
        else:
            label = f"fmr-medicaid"
        sources[label] = {
            "file": f_obj,
        }

FMR data:
- FY 2020 FMR CHIP NET EXPENDITURES.xlsx
- FY 2020 FMR NET EXPENDITURES.xlsx


---

## Notes on FMR Spreadsheets

* States, Territories, and National Totals are represented
* Medicaid file has sheets for both MAP & ADM FMR
* CHIP file has sheets for MCHIP, MCHIP 20%, and CHIP FMR
* Header row: `7`
* Shared Columns: `Service Category`, `Total Computible`, `Federal Share`, `State Share`
* Additional MAP Columns: `Federal Share Medicaid`, `Federal Share ARRA`
    * 2017-2019 includes `Federal Share BIPP`
    * 2020-2021 includes `Federal Share COVID`
* `Service Category` is text, others are numeric

## From Medicaid.gov

* Beginning with the FY 2013, the Medicaid and Administration Expenditures are separated by state into different tab within the same file labeled “MAP” and “ADM” respectively and the Medicaid CHIP expenditures were removed from the FMR Net Expenditure file and a separate file was established denoting the total of all expenditures funded by CHIP allotments.
* The C- in front of the service category denotes Medicaid CHIP expansion population expenditures funded by CHIP allotments.
* The T- in front of the service category denotes the CHIP allotment funded portion of Qualifying State expenditures authorized for certain states under section 2105(g) of the Social Security Act.

### References

* [Medicaid.gov - Expenditure Reports From MBES/CBES](https://www.medicaid.gov/medicaid/financial-management/state-expenditure-reporting-for-medicaid-chip/expenditure-reports-mbescbes/index.html)
* [American Recovery and Reinvestment Act (ARRA): Medicaid and Health Care Provisions](https://www.kff.org/medicaid/fact-sheet/american-recovery-and-reinvestment-act-arra-medicaid/)
* [Balancing Incentive Program (BIP)](https://www.medicaid.gov/medicaid/long-term-services-supports/balancing-incentive-program/balancing-incentive-program/index.html)

In [6]:
HEADER_ROW = 7
SKIP = HEADER_ROW - 1

In [7]:
mdcd_xlfile = pd.ExcelFile(sources["fmr-medicaid"]["file"])

In [8]:
mdcd_dfs = pd.read_excel(mdcd_xlfile, None, skiprows=SKIP)
print(f"MEDICAID loaded sheets: {len(mdcd_dfs)}")

MEDICAID loaded sheets: 114


In [9]:
chip_xlfile = pd.ExcelFile(sources["fmr-chip"]["file"])

In [10]:
chip_dfs = pd.read_excel(chip_xlfile, None, skiprows=SKIP)
print(f"CHIP loaded sheets: {len(chip_dfs)}")

CHIP loaded sheets: 171


In [11]:
SEPARATOR = " - "

In [12]:
def summarize_load(src_lbl:str, src_dfs:dict) -> tuple:
    fmr_types = set()
    state_terrs = set()
    for df_key in src_dfs:
        key_elems = df_key.split(SEPARATOR)
        fmr_types.add(key_elems[0].strip())
        state_terrs.add(key_elems[1].strip())
    print(f"loaded {src_lbl} types: {fmr_types}")
    print(f"loaded states/territories: {len(state_terrs)}")
    return fmr_types, state_terrs

In [13]:
mdcd_types, states = summarize_load("Medicaid", mdcd_dfs)
chip_types, states = summarize_load("CHIP", chip_dfs)

loaded Medicaid types: {'MAP', 'ADM'}
loaded states/territories: 57
loaded CHIP types: {'CHIP', 'MCHIP 20%', 'MCHIP'}
loaded states/territories: 57


In [14]:
fmr_sources = {
    "Medicaid": {
        "dataframes": mdcd_dfs,
        "datatypes": mdcd_types,
    },
    "CHIP": {
        "dataframes": chip_dfs,
        "datatypes": chip_types,
    },
}

In [15]:
TEST_STATES = ["Massachusetts"]

In [16]:
SHARED_COLS = ['Service Category', 'Total Computable', 'Federal Share', 'State Share']

In [17]:
def collect_state_fmr(state_name:str, data_types:set, src_data:dict) -> dict:
    crnt_st_data = {}
    for data_type in data_types:
        df_key = f"{data_type}{SEPARATOR}{state_name}"
        df = src_data[df_key]
        fixed_cols = [col.replace("\n ", "") for col in df.columns]
        df.columns = fixed_cols
        df = df.dropna(axis=0, how="all").reset_index(drop=True)
        print(f'<> loaded {data_type} dataframe -> rows: {df.shape[0]}, cols: {df.shape[1]}')
        print(f' - columns: {list(df.columns)}')
        drop_cols = [col for col in fixed_cols if col not in SHARED_COLS]
        if len(drop_cols):
            print(f' - dropping: {drop_cols}')
            df = df.drop(columns=drop_cols)
        created_indx = df.shape[0] - 1
        created_row = df.loc[created_indx, :]
        df.drop(created_indx, inplace=True)
        meta = created_row.to_list()[0]
        print(f' - {meta}')
        total_category = [svc for svc in df['Service Category'].to_list() if svc.find('Total') > -1][0]
        totals = df[df['Service Category'] == total_category].to_dict(orient='records')[0]
        totals.pop('Service Category')
        crnt_st_data[data_type] = {
            'dataframe': df,
            'meta': meta,
            'total_category': total_category,
            'totals': totals,
        }
    return crnt_st_data

In [18]:
def load_state_fmr(state_names:list) -> dict:
    states_fmr = {}
    for state in state_names:
        print(f'current state: {state}')
        fmr_data = {}
        for fmr, fmr_detl in fmr_sources.items():
            fmr_data.update(collect_state_fmr(
                state_name=state, 
                data_types=fmr_detl['datatypes'], 
                src_data=fmr_detl['dataframes'],
            ))
        states_fmr[states_abbrs[state]] = fmr_data
    return states_fmr

In [19]:
_ = load_state_fmr(state_names=TEST_STATES)

current state: Massachusetts
<> loaded MAP dataframe -> rows: 237, cols: 7
 - columns: ['Service Category', 'Total Computable', 'Federal Share', 'Federal Share Medicaid', 'Federal Share ARRA', 'Federal Share COVID', 'State Share']
 - dropping: ['Federal Share Medicaid', 'Federal Share ARRA', 'Federal Share COVID']
 - Created On: Wednesday, July 21, 2021 4:24 PM
<> loaded ADM dataframe -> rows: 58, cols: 4
 - columns: ['Service Category', 'Total Computable', 'Federal Share', 'State Share']
 - Created On: Wednesday, July 21, 2021 4:24 PM
<> loaded CHIP dataframe -> rows: 42, cols: 4
 - columns: ['Service Category', 'Total Computable', 'Federal Share', 'State Share']
 - Created On: Tuesday, September 21, 2021 1:20 PM
<> loaded MCHIP 20% dataframe -> rows: 94, cols: 4
 - columns: ['Service Category', 'Total Computable', 'Federal Share', 'State Share']
 - Created On: Tuesday, September 21, 2021 1:20 PM
<> loaded MCHIP dataframe -> rows: 42, cols: 4
 - columns: ['Service Category', 'Total Co